In [1]:
import sys
print(sys.executable)
sys.version

/Users/madkins/.pyenv/versions/3.8.16/envs/snowpark/bin/python3.8


'3.8.16 (default, Mar  2 2023, 19:32:24) \n[Clang 14.0.0 (clang-1400.0.29.202)]'

In [2]:
from datetime import timedelta
from getpass import getpass
import json

import numpy as np
import pandas as pd
import pyarrow as pa

from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import sproc

print('Numpy version: ' + np.__version__)
print('Pandas version: ' + pd.__version__)
print('PyArrow version: ' + pa.__version__)

#import pykx as kx
#print('PyKX version: ' + kx.__version__)
#kx.q('\c 15 120');

Numpy version: 1.23.5
Pandas version: 1.5.3
PyArrow version: 8.0.0


In [3]:
accountname = 'xe85544.east-us-2.azure'
username = 'kx'
password = 'Snowflake1'
license = 'Y8NTo4WkoyVFpNkz0oikQnrkIJbISSogiCwXxiUY/idrTFbSW9NHlidzlg1GDZYsoyeWTCzSvfkOliez/uOGvRem6fmbR9JkDEaTVpZjvVs7NltmlqSmGFvtNg2WNsuINhgs2UcXR/5Gh5vLLKMH/Nnx/DOIMyayRmMGJvEGU4i5hrJG'

connection_parameters = {
    "account": accountname,
    "user": username,
    "password": password,
    "role": "SYSADMIN",
    "database": "KX",
    "schema": "BRUNO",
    "warehouse": "KX"
}

session = Session.builder.configs(connection_parameters).create()

In [4]:
%%time

session.table('trades').count()

CPU times: user 12.3 ms, sys: 7.21 ms, total: 19.5 ms
Wall time: 142 ms


77026358

In [5]:
session.table('trades').limit(10).to_pandas()

,Time,Exchange,Symbol,Sale Condition,Trade Volume,Trade Price,Trade Stop Stock Indicator,Trade Correction Indicator,Sequence Number,Trade Id,Source of Trade,Trade Reporting Facility,Participant Timestamp,Trade Reporting Facility TRF Timestamp,Trade Through Exempt Indicator,TTime
0,105959138647808,Z,DGX,I,1,124.435,N,0,936069,52983525439551,C,None,105959138414000,None,0,2022-10-03 10:59:59.138647808
1,105959138662656,Z,DGX,I,1,124.435,N,0,936070,52983525439552,C,None,105959138424000,None,0,2022-10-03 10:59:59.138662656
2,105959138667776,Z,DGX,I,1,124.435,N,0,936071,52983525439553,C,None,105959138436000,None,0,2022-10-03 10:59:59.138667776
3,105959354907136,N,DGX,I,32,124.390,N,0,936080,52983542202673,C,None,105959354800640,None,0,2022-10-03 10:59:59.354907136
4,105959354907136,N,DGX,I,4,124.380,N,0,936081,52983542202674,C,None,105959354800640,None,0,2022-10-03 10:59:59.354907136
5,110000817275648,D,DGX,I,10,124.415,N,0,936705,71680055785803,C,T,110000816631204,110000816920273,0,2022-10-03 11:00:00.817275648
6,110004632756736,M,DGX,F I,84,124.460,N,0,937135,52983525095492,C,None,110004632718592,None,1,2022-10-03 11:00:04.632756736
7,110004632766720,M,DGX,F,116,124.460,N,0,937136,52983525095493,C,None,110004632733952,None,1,2022-10-03 11:00:04.632766720
8,110004632821760,T,DGX,I,22,124.460,N,0,937137,62879483978046,C,None,110004632472492,None,0,2022-10-03 11:00:04.632821760
9,110004632907008,N,DGX,F I,1,124.460,N,0,937138,52983542202934,C,None,110004632858368,None,1,2022-10-03 11:00:04.632907008


In [6]:
%%time

session.table('quotes').count()

CPU times: user 13.8 ms, sys: 4.81 ms, total: 18.6 ms
Wall time: 120 ms


1541781424

In [7]:
session.table('quotes').limit(10).to_pandas()

,Time,Exchange,Symbol,Bid_Price,Bid_Size,Offer_Price,Offer_Size,Quote_Condition,Sequence_Number,National_BBO_Ind,...,Retail_Interest_Indicator,Short_Sale_Restriction_Indicator,LULD_BBO_Indicator,SIP_Generated_Message_Identifier,National_BBO_LULD_Indicator,Participant_Timestamp,FINRA_ADF_Timestamp,FINRA_ADF_Market_Participant_Quote_Indicator,Security_Status_Indicator,TTime
0,150048898993597,V,AFRM,19.26,2.0,19.28,1.0,R,85494759.0,0,...,None,0,None,None,A,150048898740257,None,None,,2022-10-03 15:00:48.898993597
1,150048968168014,Q,AFRM,19.27,2.0,19.28,4.0,R,85495018.0,0,...,None,0,None,None,A,150048968155865,None,None,,2022-10-03 15:00:48.968168014
2,150048968563258,Q,AFRM,19.27,2.0,19.28,3.0,R,85495020.0,0,...,None,0,None,None,A,150048968551205,None,None,,2022-10-03 15:00:48.968563258
3,150049287852283,U,AFRM,19.27,4.0,19.28,3.0,R,85495764.0,0,...,None,0,None,None,A,150049287676895,None,None,,2022-10-03 15:00:49.287852283
4,150049289860345,Z,AFRM,19.27,1.0,19.28,2.0,R,85495766.0,0,...,None,0,None,None,A,150049289674000,None,None,,2022-10-03 15:00:49.289860345
5,150049289951846,Z,AFRM,19.26,6.0,19.28,2.0,R,85495768.0,0,...,None,0,None,None,A,150049289762000,None,None,,2022-10-03 15:00:49.289951846
6,150049290362270,V,AFRM,19.26,2.0,19.28,5.0,R,85495771.0,0,...,None,0,None,None,A,150049290112076,None,None,,2022-10-03 15:00:49.290362270
7,150049290532264,P,AFRM,19.27,3.0,19.28,5.0,R,85495773.0,0,...,None,0,None,None,A,150049290175744,None,None,,2022-10-03 15:00:49.290532264
8,150049290544849,Y,AFRM,19.22,21.0,19.31,15.0,R,85495774.0,0,...,None,0,None,None,A,150049290354000,None,None,,2022-10-03 15:00:49.290544849
9,150049290577707,K,AFRM,19.26,14.0,19.29,11.0,R,85495775.0,0,...,None,0,None,None,A,150049290389000,None,None,,2022-10-03 15:00:49.290577707


In [8]:
 %%time

def business_logic(t, q):
     from datetime import timedelta

     q['mid'] = (q['ask_price'] + q['bid_price']) / 2
     qq = q[['symbol', 'time', 'mid']]

     tt = t[['symbol', 'time']].copy()
     tt['time'] += timedelta(seconds=1)
     t['tp1s'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']

     tt = t[['symbol', 'time']].copy()
     tt['time'] += timedelta(seconds=5)
     t['tp5s'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']

     tt = t[['symbol', 'time']].copy()
     tt['time'] += timedelta(seconds=10)
     t['tp10s'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']

     tt = t[['symbol', 'time']].copy()
     tt['time'] += timedelta(minutes=1)
     t['tp1m'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']

     tt = t[['symbol', 'time']].copy()
     tt['time'] += timedelta(minutes=5)
     t['tp5m'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']

     tt = t[['symbol', 'time']].copy()
     tt['time'] += timedelta(minutes=10)
     t['tp10m'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']

     tt = t[['symbol', 'time']].copy()
     tt['time'] += timedelta(minutes=30)
     t['tp30m'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']


     tt = t[['symbol', 'time']].copy()
     tt['time'] -= timedelta(seconds=1)
     t['tm1s'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']

     tt = t[['symbol', 'time']].copy()
     tt['time'] -= timedelta(seconds=5)
     t['tm5s'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']

     tt = t[['symbol', 'time']].copy()
     tt['time'] -= timedelta(seconds=10)
     t['tm10s'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']
    
     tt = t[['symbol', 'time']].copy()
     tt['time'] -= timedelta(minutes=1)
     t['tm1m'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']

     tt = t[['symbol', 'time']].copy()
     tt['time'] -= timedelta(minutes=5)
     t['tm5m'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']

     tt = t[['symbol', 'time']].copy()
     tt['time'] -= timedelta(minutes=10)
     t['tm10m'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']

     tt = t[['symbol', 'time']].copy()
     tt['time'] -= timedelta(minutes=30)
     t['tm30m'] = pd.merge_asof(tt, qq, on='time', by='symbol')['mid']

     t['time'] = t['time'].astype('uint64')

     return t


@sproc(packages=['snowflake-snowpark-python', 'numpy', 'pandas', 'pyarrow'])
def my_pandas_procedure(session: Session) -> str:
     import json, time
    
     tstart = time.perf_counter()
     select_columns = ['"TTime"', '"Symbol"', '"Trade Volume"', '"Trade Price"']
     trades = session.table('trades').select(select_columns).order_by('"TTime"').to_pandas()
     trades.columns = ['time', 'symbol', 'volume', 'price']
     tend = time.perf_counter()
     tloadtrades = tend - tstart

     tstart = time.perf_counter()
     select_columns = ['"TTime"', '"Symbol"', '"Bid_Size"', '"Bid_Price"', '"Offer_Size"', '"Offer_Price"']
     quotes = session.table('quotes').select(select_columns).order_by('"TTime"').to_pandas()
     quotes.columns = ['time', 'symbol', 'ask_volume', 'ask_price', 'bid_volume', 'bid_price']
     tend = time.perf_counter()
     tloadquotes = tend - tstart

     tstart = time.perf_counter()
     output = business_logic(trades, quotes)
     tend = time.perf_counter()
     tpdalgo = tend - tstart
    
     tstart = time.perf_counter()
     session.write_pandas(output, 'OUTPUT_PANDAS', overwrite=True, chunk_size=1000000, compression='snappy')
     tend = time.perf_counter()
     tout = tend - tstart
        
     return json.dumps(dict(rows=len(output),
                            tloadtrades=tloadtrades,
                            tloadquotes=tloadquotes,
                            tpdalgo=tpdalgo,
 



In [9]:
# %%time

# rez = my_pandas_procedure()
# rez = json.loads(rez)
# rez

In [14]:
%%time

@sproc(packages=['snowflake-snowpark-python', 'numpy', 'pandas', 'pyarrow'],
       imports=['@kx/pykx-package.zip', '@kx/01_interval_returns.q_'])
def my_pykx_procedure(session: Session) -> str:
    import base64, json, os, shutil, sys, time, zipfile
   
    ## CUSTOM SETUP OF PyKX (won't be needed when PyKX is available on Anaconda)
    tstart = time.perf_counter()
    if not os.path.exists('/tmp/k4.lic'):
        import_dir = sys._xoptions.get("snowflake_import_directory")
        lib = 'pykx-package.zip'
        lib_path = os.path.join(import_dir, lib)
        load_path = '/tmp/pykx-package/'

        if os.path.exists('/tmp/pykx-package'):
            shutil.rmtree('/tmp/pykx-package')
            time.sleep(0)
        with zipfile.ZipFile(lib_path, 'r') as zip_ref:
            zip_ref.extractall('/tmp/pykx-package')

        sys.path.insert(0, '/tmp/pykx-package')
        
        shutil.copyfile(os.path.join(import_dir, '01_interval_returns.q_'), '/tmp/01_interval_returns.q_')

        with open('/tmp/k4.lic', 'wb') as f:
            f.write(base64.b64decode(license))
    tend = time.perf_counter()
    tpykxinstall = tend - tstart
    ## END

    tstart = time.perf_counter()
    os.environ['QHOME'] = ''
    os.environ['QLIC'] = '/tmp'

    os.environ['PYKX_LOAD_PYARROW_UNSAFE'] = '1'
    os.environ['PYKX_NOQCE'] = '1'
    os.environ['SKIP_UNDERQ'] = '1'
    os.environ['IGNORE_QHOME'] = '1'
    os.environ['QARGS'] = '--licensed'
    
    import pykx as kx
    tend = time.perf_counter()
    tpykximport = tend - tstart
    
    tstart = time.perf_counter()
    select_columns = ['"TTime"', '"Symbol"', '"Trade Volume"', '"Trade Price"']
    trades = session.table('trades').select(select_columns).where('"Symbol" = \'AAPL\'').order_by('"TTime"').to_pandas()
    trades.columns = ['time', 'symbol', 'volume', 'price']
    tend = time.perf_counter()
    tloadtrades = tend - tstart
    
    tstart = time.perf_counter()
    kx.q['trades'] = trades
    del trades
    tend = time.perf_counter()
    ttoqtrades = tend - tstart

    tstart = time.perf_counter()
    select_columns = ['"TTime"', '"Symbol"', '"Bid_Size"', '"Bid_Price"', '"Offer_Size"', '"Offer_Price"']
    quotes = session.table('quotes').select(select_columns).where('"Symbol" = \'AAPL\'').order_by('"TTime"').to_pandas()
    quotes.columns = ['time', 'symbol', 'ask_volume', 'ask_price', 'bid_volume', 'bid_price']
    tend = time.perf_counter()
    tloadquotes = tend - tstart
    
    tstart = time.perf_counter()
    kx.q['quotes'] = quotes
    del quotes
    tend = time.perf_counter()
    ttoqquotes = tend - tstart

    tstart = time.perf_counter()
    kx.q('\l /tmp/01_interval_returns.q_')  # Run a locked q script to apply the business logic
    tend = time.perf_counter()
    tqalgo = tend - tstart
    
    tstart = time.perf_counter()
    output = kx.q('t').pd()
    tend = time.perf_counter()
    ttopd = tend - tstart
    
    tstart = time.perf_counter()
    session.write_pandas(output, 'OUTPUT', overwrite=True, chunk_size=1000000, compression='snappy')
    tend = time.perf_counter()
    tout = tend - tstart
    
    return json.dumps(dict(rows=len(output),
                           tpykxinstall=tpykxinstall,
                           tpykximport=tpykximport, 
                           tloadtrades=tloadtrades,
                           tloadquotes=tloadquotes,
                           ttoqtrades=ttoqtrades,
                           ttoqquotes=ttoqquotes,
                           tqalgo=tqalgo,
                           ttopd=ttopd,
                           tout=tout))


CPU times: user 67.7 ms, sys: 11.8 ms, total: 79.5 ms
Wall time: 12.2 s


In [15]:
%%time

rez = my_pykx_procedure()
rez = json.loads(rez)
rez

CPU times: user 9.12 ms, sys: 4.96 ms, total: 14.1 ms
Wall time: 42 s


{'rows': 838794,
 'tpykxinstall': 0.1281824749999032,
 'tpykximport': 0.12329147600007673,
 'tloadtrades': 1.9326952659998824,
 'tloadquotes': 17.70989708000002,
 'ttoqtrades': 0.2467913650000355,
 'ttoqquotes': 3.4509873740000785,
 'tqalgo': 2.8336834639999324,
 'ttopd': 0.06635138199999346,
 'tout': 13.704108910999821}